In [ ]:
import pandas as pd
y = []
for i in range(1, 291):
    file = f"Maumee DL/{i}_climate_sediment.csv"
    with open(file, 'r') as f:
        df = pd.read_csv(f)
    array = df['sedyld'].tolist()
    y.extend(array)

In [ ]:
import numpy as np
y = np.array(y)
pred = np.zeros_like(y)

In [ ]:
import numpy as np

def calculate_nse(observed, predicted):
    """
    Calculate the Nash-Sutcliffe Efficiency (NSE) coefficient.
    
    Parameters:
    observed (numpy.ndarray): An array of observed values.
    predicted (numpy.ndarray): An array of predicted values.
    
    Returns:
    float: The NSE coefficient.
    """
    if observed.shape != predicted.shape:
        raise ValueError("The shape of observed and predicted arrays must be the same.")

    # Calculate the mean of the observed data
    mean_observed = np.mean(observed)
    
    # Calculate the numerator of the fraction (residual sum of squares)
    numerator = np.sum((observed - predicted) ** 2)
    
    # Calculate the denominator of the fraction (total sum of squares)
    denominator = np.sum((observed - mean_observed) ** 2)
    
    # Calculate the NSE
    nse = 1 - (numerator / denominator)
    
    return nse
print(calculate_nse(y, pred))

In [ ]:
def calculate_r_squared(observed, predicted):
    """
    Calculate the coefficient of determination, R^2.
    
    Parameters:
    observed (numpy.ndarray): An array of observed values.
    predicted (numpy.ndarray): An array of predicted values.
    
    Returns:
    float: The R^2 value.
    """
    if observed.shape != predicted.shape:
        raise ValueError("The shape of observed and predicted arrays must be the same.")

    # Calculate the mean of the observed data
    mean_observed = np.mean(observed)
    # Calculate the mean of the predicted data
    mean_predicted = np.mean(predicted)
    
    # Calculate the numerator of the R^2 formula
    numerator = np.sum((observed - mean_observed) * (predicted - mean_predicted)) ** 2
    
    # Calculate the denominator of the R^2 formula
    denominator = (np.sum((observed - mean_observed) ** 2) *
                   np.sum((predicted - mean_predicted) ** 2))
    
    # Calculate the R^2 value
    r_squared = numerator / denominator
    
    return r_squared
print(calculate_r_squared(y, pred))

In [ ]:
def calculate_pbias(observed, predicted):
    """
    Calculate the Percent Bias (PBIAS).
    
    Parameters:
    observed (numpy.ndarray): An array of observed values.
    predicted (numpy.ndarray): An array of predicted values.
    
    Returns:
    float: The PBIAS value.
    """
    if observed.shape != predicted.shape:
        raise ValueError("The shape of observed and predicted arrays must be the same.")
    
    # Calculate the numerator of the PBIAS formula
    numerator = np.sum(observed - predicted)
    
    # Calculate the denominator of the PBIAS formula
    denominator = np.sum(observed)
    
    # Calculate the PBIAS
    pbias = (numerator / denominator) * 100
    
    return pbias
print(calculate_pbias(y, pred))

In [ ]:
import pandas as pd
df = pd.read_csv("Maumee DL/276_climate_sediment.csv")

In [ ]:
print(df)

In [ ]:
df_station = pd.read_csv("Maumee DL/CrossSectionalData.csv")
print(df_station.shape)
print(type(df_station.iloc[12, 1:].values))

In [ ]:
from dataset import ClimateDatasetV2
import torch
from torch.utils.data import DataLoader
dataset = ClimateDatasetV2(split="train")

In [ ]:
print(torch.max(dataset.temporal, dim=0))
print(torch.min(dataset.temporal, dim=0))
print(torch.std(dataset.temporal, dim=0))
print(torch.mean(dataset.temporal, dim=0))



In [ ]:
import torch
# Compute the max, min, mean, and std using torch functions
max_value = torch.max(dataset.stations, dim=0)
min_value = torch.min(dataset.stations, dim=0)
mean_value = torch.mean(dataset.stations, dim=0)
std_value = torch.std(dataset.stations, dim=0)

# Create a dictionary to store the values
stats_dict = {
    'max': max_value.values,
    'min': min_value.values,
    'mean': mean_value,
    'std': std_value
}
print("Max:", max_value.values.shape)
print("Min:", min_value.values)
print("Mean:", mean_value)
print("Std:", std_value)
# Dump the dictionary into a file
torch.save(stats_dict, 'trunk_statv2.pt')


In [ ]:
print(torch.max(dataset.targets, dim=0))
print(torch.min(dataset.targets, dim=0))
print(torch.std(dataset.targets, dim=0))
print(torch.mean(dataset.targets, dim=0))

In [ ]:
from sklearn.neighbors import KernelDensity
import numpy as np
from tqdm import tqdm

a = np.random.randint(0, 100, size=[2000000, 1])
for i in tqdm(range(len(a))):
    data = a[i].reshape([1,1])
    kd = KernelDensity(kernel='gaussian', bandwidth=1).fit(data)  # should be 2D
    each_rate = np.exp(kd.score_samples(a))
    each_rate /= np.sum(each_rate)  # norm

In [ ]:
def gaussian_kernel_density_estimation(input_vector, sigma=1):
    """
    Perform Gaussian kernel density estimation on a vector.
    
    Parameters:
    input_vector (numpy.ndarray): Input vector of shape [size, feature_num].
    sigma (float): Standard deviation for the Gaussian kernel.
    
    Returns:
    numpy.ndarray: Output vector of shape [size, size].
    """
    distances = np.linalg.norm(input_vector[:, np.newaxis] - input_vector, axis=2)
    output = np.exp(-distances**2 / sigma)
    output /= np.sum(output, axis=1, keepdims=True)
    
    return output
a = np.random.randint(0, 100, size=[2000000, 1])
gaussian_kernel_density_estimation(a, 1)

In [ ]:
import torch
a = torch.tensor([[1,2,5],
                  [3,4,6]])
print(torch.sum(a, dim=1, keepdim=True))
t = a / torch.sum(a, dim=1, keepdim=True)
t

In [ ]:
idx = np.array([0,1,0])
a[idx]

In [ ]:
def knn_regression(train_features, train_targets, test_features, k):
    """
    Perform k-NN regression using PyTorch tensors.

    :param train_features: A tensor of shape (num_train_samples, num_features) containing the training data features.
    :param train_targets: A tensor of shape (num_train_samples,) or (num_train_samples, 1) containing the training data targets.
    :param test_features: A tensor of shape (num_test_samples, num_features) containing the test data features.
    :param k: The number of nearest neighbors to consider for regression.
    :return: Predicted targets for the test data.
    """
    # Ensure the input tensors are on the GPU
    # train_features = train_features
    # train_targets = train_targets.to("cuda:0")
    # test_features = test_features.to("cuda:0")

    # Calculate the pairwise distances between test points and all training points
    distances = torch.cdist(test_features, train_features)

    # Determine the indices of the k nearest neighbors (using topk to find the k smallest distances)
    _, indices = torch.topk(distances, k, largest=False, sorted=False)

    # Gather the targets of the k nearest neighbors
    nearest_targets = torch.gather(train_targets.expand(test_features.size(0), -1), 1, indices)

    # Calculate the mean target value of the k nearest neighbors
    predictions = nearest_targets.mean(dim=1)

    return predictions

import torch

# Test case 2: Multiple nearest neighbors
train_features = torch.tensor([[1, 2], [3, 4], [5, 6]]).float().cuda()
train_targets = torch.tensor([10, 20, 30]).float().cuda()
test_features = torch.tensor([[2, 3]]).float().cuda()
k = 2
expected_output = torch.tensor([15]).float().cuda()
output = knn_regression(train_features, train_targets, test_features, k)
assert torch.allclose(output, expected_output), f"Test case 2 failed: Expected {expected_output}, but got {output}"


# Test case 4: Large input tensors
train_features = torch.randn(1000, 100)
train_targets = torch.randn(1000)
test_features = torch.randn(200, 100)
k = 5
output = knn_regression(train_features, train_targets, test_features, k)
assert output.shape == (200,), f"Test case 4 failed: Expected output shape (200,), but got {output.shape}"

print("All test cases passed!")

In [ ]:
def gaussian_kernel_density_estimation(input_vector, sigma=1):
    """
    Perform Gaussian kernel density estimation on a tensor.

    Parameters:
    input_vector (torch.Tensor): Input tensor of shape [size, feature_num].
    sigma (float): Standard deviation for the Gaussian kernel.

    Returns:
    torch.Tensor: Output tensor of shape [size, size].
    """
    distances = torch.norm(input_vector.unsqueeze(1) - input_vector, dim=2)
    output = torch.exp(-distances**2 / sigma)
    output /= torch.sum(output, dim=1, keepdim=True)

    return output
input_vector = torch.tensor([1,2,3,4]).reshape(4,1).float()
print(gaussian_kernel_density_estimation(input_vector)[1])

In [ ]:
from dataset import ClimateDataset
dataset = ClimateDataset("../Maumee DL/", split='all')


In [ ]:
import random
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import pickle
import os
import pandas as pd
from torch import from_numpy

train_temporal, test_temporal = [], []
train_targets, test_targets = [], []
train_stations, test_stations = [], []
df_station = pd.read_csv(os.path.join("../Maumee DL", "CrossSectionalData.csv"))

for i in range(1, 291):
    df = pd.read_csv(os.path.join("../Maumee DL", f"{i}_climate_sediment.csv"))
    temporal_data = df.iloc[:, 1:7].values
    length = temporal_data.shape[0]
    train_temporal.append(temporal_data[:int(0.75 * length)])
    test_temporal.append(temporal_data[int(0.75 * length):])

    target = df.iloc[:, 7].values
    train_targets.append(target[:int(0.75 * length)])
    test_targets.append(target[int(0.75 * length):])
    
    station = df_station.iloc[i-1, 1:].values
    station = np.repeat(station[np.newaxis, :], temporal_data.shape[0], axis=0)
    train_stations.append(station[:int(0.75 * length)])
    test_stations.append(station[int(0.75 * length):])

train_temporal = from_numpy(np.concatenate(train_temporal, axis=0)).float()
test_temporal = from_numpy(np.concatenate(test_temporal, axis=0)).float()
train_targets = from_numpy(np.concatenate(train_targets, axis=0)).float()*1000
test_targets = from_numpy(np.concatenate(test_targets, axis=0)).float()*1000
train_stations = from_numpy(np.concatenate(train_stations, axis=0)).float()
test_stations = from_numpy(np.concatenate(test_stations, axis=0)).float()

print(train_temporal.shape, test_temporal.shape)
print(train_targets.shape, test_targets.shape)
print(train_stations.shape, test_stations.shape)

# Create a dictionary to store the train and test tensors
data_dict = {
    'train_stations': train_stations,
    'train_temporal': train_temporal,
    'train_targets': train_targets,
    'test_stations': test_stations,
    'test_temporal': test_temporal,
    'test_targets': test_targets
}

# Save the dictionary to a file
with open('../Maumee DL/train_test_data.pkl', 'wb') as f:
    pickle.dump(data_dict, f)

print("Train and test data saved successfully.")



In [ ]:
from dataset import ClimateDatasetV2B
dataset = ClimateDatasetV2B(split="train")

In [ ]:
from lstm_dataset import ClimateDatasetV2C
from torch.utils.data import DataLoader
train = ClimateDatasetV2C("../climate_new", transform="MinMaxNormalize", split="train")
stat = train.get_stats()




In [ ]:
data = ClimateDatasetV2C("../climate_new", transform="MinMaxNormalize", split="test", stats=stat)


In [ ]:
loader = DataLoader(data, batch_size=1024, shuffle=False, pin_memory=True, num_workers=18)

In [ ]:
from collections import defaultdict
station_date = defaultdict(list)
from tqdm import tqdm

for staids, dates, x, y in tqdm(loader):
    x = x[:, -1, :26]
    mask = ~(x == -1).any(dim=1)
    
    for idx, staid in enumerate(staids):
        # if staid == '1434021':
        #     print(dates[idx], mask[idx].item())
        station_date[staid].append((dates[idx], mask[idx].item()))
    

In [ ]:
for staid, items in station_date.items():
    valid_dates = []
    for date, mask in items:
        if mask:
            valid_dates.append(date)
    print(staid, len(valid_dates))

In [8]:
import pandas as pd
import numpy as np

# 读取 CSV 文件
df1 = pd.read_csv('../climate_new/01111500.csv', header=None)
df2 = pd.read_csv('/home/kh31/Xiaobo/deeponet/climate_new_pgd_0.1x_0.05_0.0125_50/01111500.csv', header=None)

# 转换为数值类型，非数值替换为 NaN
df1 = df1.apply(pd.to_numeric, errors='coerce')
df2 = df2.apply(pd.to_numeric, errors='coerce')

# 转换为 NumPy 数组
array1 = df1.to_numpy()
array2 = df2.to_numpy()

# 忽略 NaN 元素进行比较
nan_mask1 = np.isnan(array1)  # 标记 array1 中 NaN 的位置
nan_mask2 = np.isnan(array2)  # 标记 array2 中 NaN 的位置
nan_mask = nan_mask1 | nan_mask2  # 如果任意一个位置为 NaN，则忽略比较

# 比较非 NaN 元素
valid_mask = ~nan_mask1 & ~nan_mask2
difference_mask = (array1 != array2) & ~nan_mask  # 不相等且不在 NaN 掩码的位置
difference_values = np.where(difference_mask, array1 - array2, np.nan)  # 差值（保留 NaN）

total_valid_elements = np.sum(valid_mask[:,20:])
difference_mask = (array1 != array2) & valid_mask & (abs(array1 - array2) > 0.005)
different_elements_count = np.sum(difference_mask)

# 计算不同元素的占比
difference_ratio = different_elements_count / total_valid_elements
print(difference_ratio)
# 提取差异数据
rows, cols = np.where(difference_mask)  # 找到不相等的元素索引
differences = pd.DataFrame({
    'Row': rows,
    'Column': cols,
    'Value_in_File1': array1[rows, cols],
    'Value_in_File2': array2[rows, cols],
    'Difference': difference_values[rows, cols]
})

# 按绝对差值排序
differences['Abs_Difference'] = differences['Difference'].abs()
differences_sorted = differences.sort_values(by='Abs_Difference', ascending=False)

# 输出最大的差异
print("Top differences between the two files (ignoring NaN):")
print(differences_sorted[:50])



0.35291739978125447
Top differences between the two files (ignoring NaN):
          Row  Column  Value_in_File1  Value_in_File2    Difference  \
2819      138      40   217112.328125    299988.03125 -82875.703125   
2699      132      40   217112.328125    299988.03125 -82875.703125   
2779      136      40   217112.328125    299988.03125 -82875.703125   
5167      254      40   217112.328125    299988.03125 -82875.703125   
2799      137      40   217112.328125    299988.03125 -82875.703125   
5107      251      40   217112.328125    299988.03125 -82875.703125   
5207      256      40   217112.328125    299988.03125 -82875.703125   
2739      134      40   217112.328125    299988.03125 -82875.703125   
5227      257      40   217112.328125    299988.03125 -82875.703125   
2719      133      40   217112.328125    299988.03125 -82875.703125   
2759      135      40   217112.328125    299988.03125 -82875.703125   
5127      252      40   217112.328125    299988.03125 -82875.703125   
518